In [3]:
from bs4 import BeautifulSoup
import requests

def get_SP500_stocks():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S.26P_500_Component_Stocks"
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, "html5lib")

    table = soup.find_all('table')[0]
    # Exclude header row
    rows = table.find_all('tr')[1:]
    #print(len(rows))
    #print(rows[len(rows)-1])
    stocks = []
    for row in rows:
        cols = row.find_all('td')
        stocks.append(cols[0].get_text())

    return stocks
sp500 = get_SP500_stocks() 

In [20]:
startdate = '2017-10-01'
enddate = '2017-10-05'
news = []
for symbol in sp500:
    if symbol == 'AAPL':
        i = 0
        fl = 1
        while fl == 1:
            url = 'https://finance.google.com/finance/company_news?q=' + symbol + '&startdate=' + \
                   startdate + '&enddate=' + enddate + 'start=' + str(i * 10) + '&num=10'
            x = requests.get(url)
            print i
            print url
            data = x.text
            soup = BeautifulSoup(data, 'html.parser')
            #print(soup.prettify())
            quote = soup.find_all('a', {"href" : lambda L: L and L.startswith('http://news')})
            if quote:
                for row in quote:
                    try: 
                        news.append((symbol,row.next_element,row.parent.findNext('span',\
                        attrs={'class': 'date'}).next_element))
                    except: 
                        news.append((symbol,row.next_element,news[-1][1]))
            else:
                fl = 0
            i = i + 1
print news 

0
https://finance.google.com/finance/company_news?q=AAPL&startdate=2017-10-01&enddate=2017-10-05start=0&num=10
1
https://finance.google.com/finance/company_news?q=AAPL&startdate=2017-10-01&enddate=2017-10-05start=10&num=10
2
https://finance.google.com/finance/company_news?q=AAPL&startdate=2017-10-01&enddate=2017-10-05start=20&num=10
3
https://finance.google.com/finance/company_news?q=AAPL&startdate=2017-10-01&enddate=2017-10-05start=30&num=10
4
https://finance.google.com/finance/company_news?q=AAPL&startdate=2017-10-01&enddate=2017-10-05start=40&num=10
5
https://finance.google.com/finance/company_news?q=AAPL&startdate=2017-10-01&enddate=2017-10-05start=50&num=10
6
https://finance.google.com/finance/company_news?q=AAPL&startdate=2017-10-01&enddate=2017-10-05start=60&num=10
7
https://finance.google.com/finance/company_news?q=AAPL&startdate=2017-10-01&enddate=2017-10-05start=70&num=10
8
https://finance.google.com/finance/company_news?q=AAPL&startdate=2017-10-01&enddate=2017-10-05start=80&

KeyboardInterrupt: 

In [ ]:
quote2 = quote1.find_next_siblings('span', attrs={'class': 'date'})